In [4]:
from PIL import Image
import os

In [5]:
def resize_images(input_folder, output_folder, target_size):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for filename in os.listdir(input_folder):
        if filename.endswith('.JPG') or filename.endswith('.png'):
            image_path = os.path.join(input_folder, filename)
            image = Image.open(image_path)

            resized_image = image.resize(target_size)

            output_path = os.path.join(output_folder, filename)
            resized_image.save(output_path)

            image.close()

In [6]:
input_folder = "../Images/output_images/testImages/"
output_folder = "../New Dataset/"
target_size = (204, 185)

resize_images(input_folder, output_folder, target_size)